我们可以使用``torch.nn``包构建神经网络。

现在您已经了解了``autograd``，``nn``依赖于``autograd``来定义模型并微分它们。``nn.Module``包含``layers``，以及返回``output``的``forward(input)``方法。

神经网络的典型训练过程如下：
* 定义具有一些可学习参数（或权重）的神经网络
* 迭代输入数据集
* 通过网络处理输入    
* 计算损失（output距离正确的多远） 
* 将梯度反向传播给网络参数
* 通常使用简单的更新规则更新网络权重：``weight = weight - learning_rate * gradient``

## Define the network

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x 
    
    def num_flat_features(self, x):
        size = x.size()[1:]    # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features    #　将　通道*长*宽，展平

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


我们已经定义``forward``函数，而`backward`函数（计算梯度）会自动被定义，因为我们使用`autograd`。现在我们可以在`forward`函数中使用任何Tensor操作。

模型的可学习参数由`net.parameters()`返回。

In [2]:
params = list(net.parameters())
print(len(params))
print(params[0].size())    # conv1's .weight，　6 x 1 x 5 x 5 

10
torch.Size([6, 1, 5, 5])


让我们尝试一个随机的32x32输入。 注意：此网络（LeNet）的预期输入大小为32x32。要在MNIST数据集上使用此网络，请将数据集中的图像大小调整为32x32。

In [3]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0531,  0.0635,  0.1105, -0.0892, -0.1088,  0.0610, -0.0553,  0.0929,
         -0.0130,  0.0592]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [4]:
net.zero_grad()
out.backward(torch.randn(1, 10))

<div class="alert alert-info"><h4>注意</h4><p>

``torch.nn`` 仅仅支持小批量数据. 整个``torch.nn``包仅仅支持mini-batch的样本，而不是单个样本。

比如，``nn.Conv2d`` 将接受一个４D的Tensor： ``nSamples x nChannels x Height x Width``.

如果仅有一个样本, 就使用``input.unsqueeze(0)`` 添加一个fake batch dimension
</p></div>

## Loss Function

损失函数采用(output，target）输入对，并计算我们的预测output与target之间的差距。

``nn``包下有一些不同的[损失函数](https://pytorch.org/docs/stable/nn.html)。一个简单的损失函数是：`nn.MSELoss`，它计算output和target之间的均方误差。

In [11]:
output = net(input)
target = torch.randn(10)    # torch.Size([10])
target = target.view(1, -1) # make it the same shape as output：torch.Size([1, 10])
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.2495, grad_fn=<MseLossBackward>)


现在，如果我们跟随``loss``往反向传播，使用其``.grad_fn``属性，我们将看到如下所示的计算图：

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t.(with regard to) the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

In [12]:
print(loss.grad_fn)    # MSELoss
print(loss.grad_fn.next_functions[0][0])    # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])    # Relu

## Backprop

为了反向传播(backpropagate)错误，我们所要做的就是``loss.backward()``。我们需要清除现有梯度，否则梯度将累积到现有梯度。

我们调用``loss.backward()``，观察一下conv1's 偏置梯度，在反向传播前和反向传播后的变化

In [13]:
net.zero_grad()    # zeroes the gradient buffers of all parameters

print("conv1.bais.grad before backward")
print(net.conv1.bias.grad)

loss.backward()

print("conv1.bais.grad after backward")
print(net.conv1.bias.grad)

conv1.bais.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bais.grad after backward
tensor([ 0.0060,  0.0053, -0.0098,  0.0054,  0.0052, -0.0090])


**Read Later:**

  The neural network package contains various modules and loss functions
  that form the building blocks of deep neural networks. A full list with
  documentation is [here](https://pytorch.org/docs/nn).


## Update the weights

实践中使用的最简单的更新规则是随机梯度下降（Stochastic Gradient Descent，SGD）：

<center>``weight = weight - learning_rate * gradient``</center>

In [14]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

但是，当您使用神经网络时，您希望使用各种不同的更新规则，例如`SGD`，`Nesterov-SGD`，`Adam`，`RMSProp`等。为了实现这一点，我们构建了一个小包：`torch.optim`，它实现了所有这些方法。 使用它非常简单：

In [15]:
import torch.optim as optim

# create your optimizer 
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop
optimizer.zero_grad()    # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

Observe how gradient buffers had to be manually set to zero using
``optimizer.zero_grad()``. This is because gradients are accumulated
as explained in `Backprop` section.